In [3]:
import tensorflow as tf
import numpy as np
import os 
import cv2
import random
from tqdm import tqdm
from tensorflow.keras.layers import LSTM, Bidirectional, Conv2D, Dense, Flatten, MaxPooling2D, TimeDistributed, Reshape
from tensorflow.keras.models import Sequential
import time
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D, GlobalAveragePooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow import keras

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from random import shuffle
from sklearn.model_selection import KFold
from tensorflow.keras import optimizers
from IPython.display import Image, display
import matplotlib.cm as cm

In [14]:
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.9  # 0.6 sometimes works better for folks
keras.backend.tensorflow_backend.set_session(tf.Session(config=config))

AttributeError: module 'tensorflow' has no attribute 'ConfigProto'

In [12]:
IMG_SIZE = 224
decode_predictions = keras.applications.inception_v3.decode_predictions


In [15]:
base_model = tf.keras.applications.InceptionV3(
    include_top=False,
    weights="imagenet",
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
)

base_model.trainable = False
base_model.summary()



Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d_188 (Conv2D)             (None, 111, 111, 32) 864         input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_188 (BatchN (None, 111, 111, 32) 96          conv2d_188[0][0]                 
__________________________________________________________________________________________________
activation_188 (Activation)     (None, 111, 111, 32) 0           batch_normalization_188[0][0]    
_______________________________________________________________________________________

In [20]:
flat1 = Flatten()(base_model.output)
output = Dense(2, activation= 'softmax')(flat1)

In [21]:
model = Model(inputs= base_model.input, outputs= [base_model.get_layer('conv2d_281').output, output])

In [5]:
add_model = Sequential()
add_model.add(base_model)
add_model.add(Flatten())
add_model.add(Dense(2, activation='sigmoid'))

model = add_model
# model.compile(loss='categorical_crossentropy', optimizer = optimizers.SGD(lr=LR, momentum = 0.9), metrics = ['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Functional)    (None, 5, 5, 2048)        21802784  
_________________________________________________________________
flatten (Flatten)            (None, 51200)             0         
_________________________________________________________________
dense (Dense)                (None, 2)                 102402    
Total params: 21,905,186
Trainable params: 102,402
Non-trainable params: 21,802,784
_________________________________________________________________


In [6]:
model.get_layer('inception_v3').summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 111, 111, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 111, 111, 32) 0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [4]:
model_tf = load_model('./movie-model-softmax2.h5')

In [4]:
model_tf.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Functional)    (None, 5, 5, 2048)        21802784  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 2)                 4098      
Total params: 21,806,882
Trainable params: 4,098
Non-trainable params: 21,802,784
_________________________________________________________________


In [5]:
model_tf.get_layer('inception_v3').summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 111, 111, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 111, 111, 32) 0           batch_normalization[0][0]        
_______________________________________________________________________________________

In [26]:
last_conv_layer_name = 'mixed10'

PYIMAGESEARCH GRADCAM CLASS

In [10]:
from tensorflow.keras.models import Model
gradModel = Model(inputs=[model_tf.input], outputs=[model_tf.get_layer('inception_v3').output])

ValueError: Graph disconnected: cannot obtain value for tensor KerasTensor(type_spec=TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_1'), name='input_1', description="created by layer 'input_1'") at layer "conv2d". The following previous layers were accessed without issue: []

In [10]:
preds = model_tf.predict(img_array)

In [18]:
print(preds[0][0])
print(preds)

0.94830763
[[0.94830763 0.05169239]]


In [19]:
vio = model_tf.output[:, np.argmax(preds[0][0])]
vio

<KerasTensor: shape=(None,) dtype=float32 (created by layer 'tf.__operators__.getitem_2')>

In [20]:
last_conv_layer = model_tf.get_layer('inception_v3').get_layer('conv_93')

In [25]:
from tensorflow.keras import backend as K
with tf.GradientTape() as tape:
    vio = model_tf.output[:, np.argmax(preds[0][0])]

grads = tape.gradient(vio, last_conv_layer.output)

AttributeError: 'KerasTensor' object has no attribute '_id'

In [8]:
def get_img_array(img_path, size):
    # `img` is a PIL image of size 299x299
    img = cv2.resize(cv2.imread(img_path), (size, size))
    # `array` is a float32 Numpy array of shape (299, 299, 3)
    array = np.array(img)
    # We add a dimension to transform our array into a "batch"
    # of size (1, 299, 299, 3)
    array = np.expand_dims(array, axis=0)
    return array


def make_gradcam_heatmap(img_array, model, base_model, last_conv_layer_name, pred_index=None):
    # First, we create a model that maps the input image to the activations
    # of the last conv layer as well as the output predictions
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer('inception_v3').get_layer(last_conv_layer_name).output, model.output]
    )

    # Then, we compute the gradient of the top predicted class for our input image
    # with respect to the activations of the last conv layer
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    # This is the gradient of the output neuron (top predicted or chosen)
    # with regard to the output feature map of the last conv layer
    grads = tape.gradient(class_channel, last_conv_layer_output)

    # This is a vector where each entry is the mean intensity of the gradient
    # over a specific feature map channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # We multiply each channel in the feature map array
    # by "how important this channel is" with regard to the top predicted class
    # then sum all the channels to obtain the heatmap class activation
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # For visualization purpose, we will also normalize the heatmap between 0 & 1
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

In [9]:
img_array = preprocess_input(get_img_array('./Test_movie/11.jpg', size=IMG_SIZE))

In [10]:
preds = model_tf.predict(img_array)

UnknownError:  Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node sequential/inception_v3/conv2d/Conv2D (defined at <ipython-input-9-5704900319b4>:1) ]] [Op:__inference_predict_function_8671]

Function call stack:
predict_function


In [ ]:
print(preds)

In [23]:
print(model_tf.get_layer('inception_v3').output)
print(model_tf.input)
print(model_tf.output)

KerasTensor(type_spec=TensorSpec(shape=(None, 5, 5, 2048), dtype=tf.float32, name=None), name='mixed10/concat:0', description="created by layer 'mixed10'")
KerasTensor(type_spec=TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='inception_v3_input'), name='inception_v3_input', description="created by layer 'inception_v3_input'")
KerasTensor(type_spec=TensorSpec(shape=(None, 2), dtype=tf.float32, name=None), name='dense/Softmax:0', description="created by layer 'dense'")


In [13]:
grad_model = tf.keras.models.Model(
        [model_tf.inputs], [model_tf.get_layer('inception_v3').get_layer(last_conv_layer_name).output, model_tf.output]
    )

ValueError: Graph disconnected: cannot obtain value for tensor KerasTensor(type_spec=TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_1'), name='input_1', description="created by layer 'input_1'") at layer "conv2d". The following previous layers were accessed without issue: []

In [11]:
tf.config.list_physical_devices()


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [ ]:
img_array = preprocess_input(get_img_array('./Test_movie/11.jpg', size=img_size))


# Print what the top predicted class is
preds = model.predict(img_array)
print("Predicted:", decode_predictions(preds, top=1)[0])

# Generate class activation heatmap


# Display heatmap
plt.matshow(heatmap)
plt.show()